# Double Pendulum Equations of Motion  
## Lagrangian Formulation

![](Image_files/Massive_rods.png)

#### Compound Double Pendulum System: 
Figure shows a planar, double compound pendulum hinged at $O$, represented by two mass-carrying uniform rods, connected in series via a frictionless hinge and allowed to move in the $(x,y)$-plane.

- **Massive Rods**: The rods $OP_1$ and $P_1P_2$ possess non-zero masses $M_1$ and $M_2$.
  - $OP_1$ performs rotation about the fixed axis
  - $P_1P_2$ is in plane motion. 
- **Moment of Inertia**: Signifying resistance to rotational acceleration about the $_{\text{CM}}$.
- **Centres of Mass**: Labelled $C_1$ and $C_2$

The system's state is uniquely determined by two rotational angles $\theta_1$ and $\theta_2$, offering two degrees of freedom.

The parallel axis theorem is used to determine the moment of inertia of a body about any axis, given the moment of inertia of the body about a parallel axis that passes through its center of mass and the perpendicular distance between the two axes.

1. Calculate the moment of inertia $I_{\text{CM}}$ about the axis through the center of mass using the standard formula for the given geometry of the body.
2. Measure the distance $d$ from the $\text{CM}$ to the new axis about which you want to find the moment of inertia.
3. Apply the theorem: $I = I_{\text{CM}} + ML^2$, where:
   - $I$ is the moment of inertia about the new axis,
   - $I_{\text{CM}}$ is the moment of inertia about the $\text{CM}$ axis,
   - $M$ is the mass of the body,
   - $L$ is the distance between the two parallel axes.

This allows the calculation of the rotational kinetic energy $T_{\text{rot}} = \frac{1}{2}I\omega^2$, where $\omega$ is the angular velocity of the body about the new axis.


In [2]:
import sympy as sp

In [3]:
# Declare variables & constants
t = sp.Symbol("t") 
l1, l2, M1, M2, g = sp.symbols('l1 l2 M1 M2 g', real=True, positive=True)
R1, R2 = sp.symbols('R1 R2', real=True, positive=True)
theta1 = sp.Function('theta1')(t)
theta2 = sp.Function('theta2')(t)

In [4]:
omega1 = sp.diff(theta1, t)
omega2 = sp.diff(theta2, t)

----
&nbsp;
## Define Functions

In [5]:
def trans_kinetic_energy(m, dx, dy, dz):
    T = sp.Rational(1,2)*m*(dx**2 + dy**2 + dz**2)
    return T

In [6]:
def potential_energy(m, g, h):
    V = m*g*h
    return V

In [7]:
def moment_of_inertia(M, L, model='uniform', R=None):
    """
    Calculate the moment of inertia for a rod based on the specified model.
    Can be 'uniform' for a uniform rod or 'cylindrical' for a cylindrical rod.
    The default is 'uniform'.
        
    Returns:
    The moment of inertia of the rod based on the specified model.
        
    Raises:
    ValueError
        If the 'cylindrical' model is selected and no radius R is provided.  
    """
    if model == 'uniform':
        # The moment of inertia for a thin rod
        I_cm = sp.Rational(1, 12) * M * L**2
        # Applying parallel axis theorem
        I_end = I_cm + M * (L/2)**2
        return I_end
    elif model == 'cylindrical':
        if R is None:
            raise ValueError("Radius R must be provided for the cylindrical model.")
        # The moment of inertia for a solid cylinder
        I_cm = sp.Rational(1, 4) * M * R**2 + sp.Rational(1, 12) * M * L**2
        # Applying the parallel axis theorem 
        I_end = I_cm + M * (L / 2)**2
        return I_end

In [8]:
def rotational_kinetic_energy(M, L, omega, model='uniform', R=None):
    I = moment_of_inertia(M, L, model, R) 
    T_rot = sp.Rational(1, 2) * I * omega**2
    return T_rot

In [9]:
def form_lagrangian(theta1, theta2, l1, l2, M1, M2, g, R1=None, R2=None, model='uniform'):
    """
    The function computes the Lagrangian by determining the translational and rotational kinetic energies, 
    as well as the potential energy, of two rods in a pendulum system. 
    It supports 'uniform' and 'cylindrical' models. (currently)

    Returns the symbolic expression for the Lagrangian of the system.
    """
    # Calculate positions of the center of mass
    x1 = l1 / 2 * sp.sin(theta1)
    y1 = -l1 / 2 * sp.cos(theta1)
    x2 = x1 + l2 / 2 * sp.sin(theta2)
    y2 = y1 - l2 / 2 * sp.cos(theta2)

    # Calculate velocities
    xdot1 = sp.diff(x1, t)
    ydot1 = sp.diff(y1, t)
    xdot2 = sp.diff(x2, t)
    ydot2 = sp.diff(y2, t)
    
    # Define angular velocity
    omega_1 = sp.diff(theta1, t)
    omega_2 = sp.diff(theta2, t)

    # Calculate energies using the previously defined functions
    T1_trans = trans_kinetic_energy(M1, xdot1, ydot1, 0)
    V1 = potential_energy(M1, g, y1)
    T1_rot = rotational_kinetic_energy(M1, l1, omega_1, model, R1)

    T2_trans = trans_kinetic_energy(M2, xdot2, ydot2, 0)
    V2 = potential_energy(M2, g, y2)
    T2_rot = rotational_kinetic_energy(M2, l2, omega_2, model, R2)

    # Form the Lagrangian
    T = sp.simplify(T1_trans + T1_rot + T2_trans + T2_rot)
    V = sp.simplify(V1 + V2)
    L = T - V

    return L

----
&nbsp;
## Form Lagrangian

`Uniform rod` model

In [10]:
L_uniform = form_lagrangian(theta1, theta2, l1, l2, M1, M2, g, model='uniform')
display(L_uniform)

7*M1*l1**2*Derivative(theta1(t), t)**2/24 + M2*l1**2*Derivative(theta1(t), t)**2/8 + M2*l1*l2*cos(theta1(t) - theta2(t))*Derivative(theta1(t), t)*Derivative(theta2(t), t)/4 + 7*M2*l2**2*Derivative(theta2(t), t)**2/24 + g*(M1*l1*cos(theta1(t)) + M2*l1*cos(theta1(t)) + M2*l2*cos(theta2(t)))/2

`cylindrical_rod` model

In [11]:
L_cylindrical = form_lagrangian(theta1, theta2, l1, l2, M1, M2, g, R1, R2, model='cylindrical')
display(L_cylindrical)

M1*R1**2*Derivative(theta1(t), t)**2/8 + 7*M1*l1**2*Derivative(theta1(t), t)**2/24 + M2*R2**2*Derivative(theta2(t), t)**2/8 + M2*l1**2*Derivative(theta1(t), t)**2/8 + M2*l1*l2*cos(theta1(t) - theta2(t))*Derivative(theta1(t), t)*Derivative(theta2(t), t)/4 + 7*M2*l2**2*Derivative(theta2(t), t)**2/24 + g*(M1*l1*cos(theta1(t)) + M2*l1*cos(theta1(t)) + M2*l2*cos(theta2(t)))/2

----
&nbsp;
## System of Euler-Lagrange Equations: 
For each generalised coordinate $\theta_i$

$$\frac{\text{d}}{\text{d}t}\left(\frac{\partial L}{\partial \dot{\theta}_i}\right)-\frac{\partial L}{\partial \theta_i}=0$$

In [12]:
def euler_lagrange_system(L, q1, q2, model='uniform'):
    """
    Computes the Euler-Lagrange equations for a double pendulum system with two generalized coordinates.

    Parameters
    ----------
    q1 : sympy.Function
        The first generalized coordinate as a function of time.
    q2 : sympy.Function
        The second generalized coordinate as a function of time.
    model : str, optional
        The model type of the pendulum system, either 'uniform' or 'cylindrical'. Default is 'uniform'.

    Returns
    -------
    tuple of sympy.Eq
        A tuple containing two sympy.Eq objects representing the simplified Euler-Lagrange equations
        for the two generalized coordinates.

    Raises
    ------
    ValueError
        If an invalid model type is provided.

    Notes
    -----
    The function assumes that the Lagrangians (L_uniform and L_cylindrical) for both model types
    are precomputed and accessible in the scope of this function.
    """
    # Derivatives of the generalized coordinates
    q1_dot = sp.diff(q1, t)
    q2_dot = sp.diff(q2, t)

    # Euler-Lagrange equations for q1
    partial_L_partial_q1 = sp.diff(L, q1)
    partial_L_partial_q1_dot = sp.diff(L, q1_dot)
    EL_q1 = sp.diff(partial_L_partial_q1_dot, t) - partial_L_partial_q1

    # Euler-Lagrange equations for q2
    partial_L_partial_q2 = sp.diff(L, q2)
    partial_L_partial_q2_dot = sp.diff(L, q2_dot)
    EL_q2 = sp.diff(partial_L_partial_q2_dot, t) - partial_L_partial_q2

    # Apply specific simplification factors based on the model
    if model == 'uniform':
        EL_q1 = EL_q1 * (12 / l1)
        EL_q2 = EL_q2 * (12 / (M2 * l2))
    elif model == 'cylindrical':
        EL_q1 = EL_q1 * 2
        EL_q2 = EL_q2 * (4 / M2)

    # Simplify the equations
    EL_q1_simplified = sp.simplify(EL_q1)
    EL_q2_simplified = sp.simplify(EL_q2)
    
    eqn1 = sp.Eq(EL_q1_simplified, 0)
    eqn2 = sp.Eq(EL_q2_simplified, 0)

    return eqn1, eqn2

`uniform_rod` model

In [13]:
eqn1_u, eqn2_u = euler_lagrange_system(L_uniform, theta1, theta2, model='uniform')
uniform_system = [eqn1_u, eqn2_u]
for i, eqn in enumerate(uniform_system):
    print(f"eqn{i+1}\n{type(eqn)}")
    display(eqn)

eqn1
<class 'sympy.core.relational.Equality'>


Eq(6*M1*g*sin(theta1(t)) + 7*M1*l1*Derivative(theta1(t), (t, 2)) + 6*M2*g*sin(theta1(t)) + 3*M2*l1*Derivative(theta1(t), (t, 2)) + 3*M2*l2*sin(theta1(t) - theta2(t))*Derivative(theta2(t), t)**2 + 3*M2*l2*cos(theta1(t) - theta2(t))*Derivative(theta2(t), (t, 2)), 0)

eqn2
<class 'sympy.core.relational.Equality'>


Eq(6*g*sin(theta2(t)) - 3*l1*sin(theta1(t) - theta2(t))*Derivative(theta1(t), t)**2 + 3*l1*cos(theta1(t) - theta2(t))*Derivative(theta1(t), (t, 2)) + 7*l2*Derivative(theta2(t), (t, 2)), 0)

`cylindrical_rod` model

In [14]:
eqn1_c, eqn2_c = euler_lagrange_system(L_cylindrical, theta1, theta2, model='cylindrical')
cylindrical_system = [eqn1_c, eqn2_c]
for i, eqn in enumerate(cylindrical_system):
    print(f"eqn{i+1}\n{type(eqn)}\n")
    print(eqn)
    display(eqn)

eqn1
<class 'sympy.core.relational.Equality'>

Eq(M1*R1**2*Derivative(theta1(t), (t, 2))/2 + M1*g*l1*sin(theta1(t)) + 7*M1*l1**2*Derivative(theta1(t), (t, 2))/6 + M2*g*l1*sin(theta1(t)) + M2*l1**2*Derivative(theta1(t), (t, 2))/2 + M2*l1*l2*sin(theta1(t) - theta2(t))*Derivative(theta2(t), t)**2/2 + M2*l1*l2*cos(theta1(t) - theta2(t))*Derivative(theta2(t), (t, 2))/2, 0)


Eq(M1*R1**2*Derivative(theta1(t), (t, 2))/2 + M1*g*l1*sin(theta1(t)) + 7*M1*l1**2*Derivative(theta1(t), (t, 2))/6 + M2*g*l1*sin(theta1(t)) + M2*l1**2*Derivative(theta1(t), (t, 2))/2 + M2*l1*l2*sin(theta1(t) - theta2(t))*Derivative(theta2(t), t)**2/2 + M2*l1*l2*cos(theta1(t) - theta2(t))*Derivative(theta2(t), (t, 2))/2, 0)

eqn2
<class 'sympy.core.relational.Equality'>

Eq(R2**2*Derivative(theta2(t), (t, 2)) + 2*g*l2*sin(theta2(t)) - l1*l2*sin(theta1(t) - theta2(t))*Derivative(theta1(t), t)**2 + l1*l2*cos(theta1(t) - theta2(t))*Derivative(theta1(t), (t, 2)) + 7*l2**2*Derivative(theta2(t), (t, 2))/3, 0)


Eq(R2**2*Derivative(theta2(t), (t, 2)) + 2*g*l2*sin(theta2(t)) - l1*l2*sin(theta1(t) - theta2(t))*Derivative(theta1(t), t)**2 + l1*l2*cos(theta1(t) - theta2(t))*Derivative(theta1(t), (t, 2)) + 7*l2**2*Derivative(theta2(t), (t, 2))/3, 0)

#### Which are our coupled 2nd-order differential equations of motion

----
&nbsp;
#### Rewrite E-L equations in form;

$$
\begin{aligned}
& \ddot{\theta}_1+\alpha_1\left(\theta_1, \theta_2\right) \ddot{\theta}_2=f_1\left(\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2\right) \\
& \ddot{\theta}_2+\alpha_2\left(\theta_1, \theta_2\right) \ddot{\theta}_1=f_2\left(\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2\right)
\end{aligned}
$$

  

Extract equations from system to avoid name space conflicts

In [15]:
system = uniform_system
eq1, eq2 = system[0], system[1]

#### Isolate the second derivative coefficients

In [16]:
def isolate_terms(equation):
    """
    Returns the second derivative terms isolated from the given equation.
    
    Parameters:
        equation (sympy.Eq): The equation from which to isolate the second derivative terms.
    
    Returns:
        list: A list containing the isolated second derivative terms.
    """
    # Define derivative terms
    theta1ddot = sp.diff(theta1, t, 2)
    theta2ddot = sp.diff(theta2, t, 2)
    
    terms = []

    try:
        th1 = sp.Eq(theta1ddot, sp.solve(equation, theta1ddot)[0])
        rhs_eq = th1.rhs
        alpha1 = sp.together(rhs_eq).as_numer_denom()[1]
        eq_new = sp.Eq(th1.lhs * alpha1, th1.rhs * alpha1)
        theta1_second = eq_new.lhs
        terms.append(theta1_second)
    except IndexError:
        pass

    try:
        th2 = sp.Eq(theta2ddot, sp.solve(equation, theta2ddot)[0])
        rhs_eq = th2.rhs
        alpha2 = sp.together(rhs_eq).as_numer_denom()[1]
        eq_new = sp.Eq(th2.lhs * alpha2, th2.rhs * alpha2)
        theta2_second = eq_new.lhs
        terms.append(theta2_second)
    except IndexError:
        pass

    return terms

In [17]:
def simplify_system_equations(eq1, eq2, model='uniform'):
    """
    Simplifies the system of equations based on the specified model.
    
    Parameters:
    eq1, eq2 : sympy.Eq
        The Euler-Lagrange equations of the system.
    model : str, optional
        The model type of the pendulum system ('uniform' or 'cylindrical'). Default is 'uniform'.
    
    Returns:
    tuple of sympy.Eq
        The simplified equations for the system.
    """
    if model == 'uniform':
        # Isolate second derivatives and simplify equations for eq1
        second_derivatives_1 = isolate_terms(eq1)
        lhs_1 = sum(second_derivatives_1)
        rhs_1 = sp.simplify(eq1.lhs - lhs_1) 
        eqn1 = sp.Eq(lhs_1, rhs_1)

        # Isolate second derivatives and simplify equations for eq2
        second_derivatives_2 = isolate_terms(eq2)
        lhs_2 = sum(second_derivatives_2)
        rhs_2 = sp.simplify(eq2.lhs - lhs_2)
        eqn2 = sp.Eq(lhs_2, rhs_2)

        # Simplify eqn1
        eqn1_lhs = sp.simplify(eqn1.lhs / (l1 * (7*M1 + 3*M2)))
        eqn1_rhs = sp.simplify(eqn1.rhs / (l1 * (7*M1 + 3*M2)))
        eqn1 = sp.Eq(eqn1_lhs, eqn1_rhs)

        # Simplify eqn2
        eqn2_lhs = sp.simplify(eqn2.lhs / (7 * l2))
        eqn2_rhs = sp.simplify(eqn2.rhs / (7 * l2))
        eqn2 = sp.Eq(eqn2_lhs, eqn2_rhs)

        return eqn1, eqn2
    
    elif model == 'cylindrical':
        # Define second-order derivative terms
        theta1_ddot = sp.diff(theta1, t, 2)
        theta2_ddot = sp.diff(theta2, t, 2)

        # Collect both second derivatives in eq1 and isolate other terms
        eq1_lhs_collected = sp.collect(eq1.lhs, (theta1_ddot, theta2_ddot))
        eqn1_rhs = sp.sympify(M1*g*l1*sp.sin(theta1) + M2*g*l1*sp.sin(theta1) + M2*l1*l2*sp.sin(theta1 - theta2)*sp.diff(theta2, t)**2/2)
        eqn1_lhs = eq1_lhs_collected - eqn1_rhs
        eq1_simp = sp.Eq(eqn1_lhs, -eqn1_rhs)

        # Collect both second derivatives in eq2 and isolate other terms
        eq2_lhs_collected = sp.collect(eq2.lhs, (theta1_ddot, theta2_ddot))
        eqn2_rhs = sp.sympify(2*g*l2*sp.sin(theta2) - l1*l2*sp.sin(theta1 - theta2)*sp.diff(theta1, t)**2)
        eqn2_lhs = eq2_lhs_collected - eqn2_rhs
        eq2_simp = sp.Eq(eqn2_lhs, eqn2_rhs)
        
        # Simplify further
        eqn1_lhs = sp.simplify(eq1_simp.lhs / (M1*R1**2/2 + 7*M1*l1**2/6 + M2*l1**2/2))
        eqn1_rhs = sp.simplify(eq1_simp.rhs / (M1*R1**2/2 + 7*M1*l1**2/6 + M2*l1**2/2))
        eqn1 = sp.Eq(eqn1_lhs, eqn1_rhs)
       
        eqn2_lhs = sp.simplify(eq2_simp.lhs / (R2**2 + 7*l2**2/3))
        eqn2_rhs = sp.simplify(eq2_simp.rhs / (R2**2 + 7*l2**2/3))
        eqn2 = sp.Eq(eqn2_lhs, eqn2_rhs)

        return eqn1, eqn2

In [18]:
eqn1, eqn2 = simplify_system_equations(eq1, eq2, model='uniform')

In [19]:
system = [eqn1, eqn2]
for i, eqn in enumerate(system):
    print(f"eqn{i+1}\n{type(eqn)}")
    display(eqn)

eqn1
<class 'sympy.core.relational.Equality'>


Eq((3*M2*l2*cos(theta1(t) - theta2(t))*Derivative(theta2(t), (t, 2)) + l1*(7*M1 + 3*M2)*Derivative(theta1(t), (t, 2)))/(l1*(7*M1 + 3*M2)), 3*(2*M1*g*sin(theta1(t)) + 2*M2*g*sin(theta1(t)) + M2*l2*sin(theta1(t) - theta2(t))*Derivative(theta2(t), t)**2)/(l1*(7*M1 + 3*M2)))

eqn2
<class 'sympy.core.relational.Equality'>


Eq(3*l1*cos(theta1(t) - theta2(t))*Derivative(theta1(t), (t, 2))/(7*l2) + Derivative(theta2(t), (t, 2)), 3*(2*g*sin(theta2(t)) - l1*sin(theta1(t) - theta2(t))*Derivative(theta1(t), t)**2)/(7*l2))

----
&nbsp;
## Extract coefficients from above equations

In [20]:
def extract_coefficient(equation, derivative_term):
    """
    Extracts the coefficient, including the denominator, of the specified derivative term from the left-hand side of the equation.

    Parameters:
        equation (sympy.Expr): The equation to extract the coefficient from.
        derivative_term (sympy.Expr): The derivative term to extract the coefficient of.

    Returns:
        coeff (sympy.Expr): The coefficient, including the denominator, of the derivative term in the equation.
    """
    lhs_parts = sp.fraction(equation.lhs)
    lhs_coeff_term = lhs_parts[0]
    lhs_denominator_term = lhs_parts[1]
    coeff = lhs_coeff_term.coeff(derivative_term) / lhs_denominator_term

    return coeff

We can now rewrite in the desired form

$$
\begin{aligned}
& \ddot{\theta}_1+\alpha_1\left(\theta_1, \theta_2\right) \ddot{\theta}_2=f_1\left(\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2\right) \\
& \ddot{\theta}_2+\alpha_2\left(\theta_1, \theta_2\right) \ddot{\theta}_1=f_2\left(\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2\right)
\end{aligned}
$$

In [21]:
# Let alpha1 be 2nd derivative coefficient of theta2 from eqn1
alpha1 = extract_coefficient(eqn1, sp.diff(theta2, t, 2))
print(type(alpha1))
display(alpha1)

<class 'sympy.core.mul.Mul'>


3*M2*l2*cos(theta1(t) - theta2(t))/(l1*(7*M1 + 3*M2))

In [22]:
# Let alpha2 be 2nd derivative coefficient of theta1 from eqn2
alpha2 = extract_coefficient(eqn2, sp.diff(theta1, t, 2))
print(type(alpha2))
display(alpha2)

<class 'sympy.core.mul.Mul'>


3*l1*cos(theta1(t) - theta2(t))/(7*l2)

Declare RHS as expressions

In [23]:
function_1 = eqn1.rhs
print(type(function_1))
display(function_1)

<class 'sympy.core.mul.Mul'>


3*(2*M1*g*sin(theta1(t)) + 2*M2*g*sin(theta1(t)) + M2*l2*sin(theta1(t) - theta2(t))*Derivative(theta2(t), t)**2)/(l1*(7*M1 + 3*M2))

In [24]:
function_2 = eqn2.rhs
print(type(function_2))
display(function_2)

<class 'sympy.core.mul.Mul'>


3*(2*g*sin(theta2(t)) - l1*sin(theta1(t) - theta2(t))*Derivative(theta1(t), t)**2)/(7*l2)

----
&nbsp;
## Create Matrix Equations

In [25]:
def create_matrix_equation(theta1dd_coeff, theta2dd_coeff, rhs_1, rhs_2):
    """
    This function constructs a matrix A based on symbolic functions alpha1 and alpha2, which represent
    coefficients related to the system's inertia, and solves the equation A*[theta1'', theta2''] = -[f1, f2]
    for the angular accelerations.

    Parameters:
    theta1dd_coeff : symbolic expression
        The coefficient for theta1's second derivative, as derived from the system's dynamics.
    theta2dd_coeff : symbolic expression
        The coefficient for theta2's second derivative, as derived from the system's dynamics.
    rhs_1 : symbolic expression
        The function representing the external force or torque acting on theta1.
    rhs_2 : symbolic expression
        The function representing the external force or torque acting on theta2.

    Returns:
    sympy.Eq
        The symbolic equation representing the system, with the specified coefficients and forces/torques substituted in.
    """
    # Define the symbolic functions for alpha1 and alpha2
    alpha1 = sp.Function('alpha1')(theta1, theta2)
    alpha2 = sp.Function('alpha2')(theta1, theta2)
    
    # Define the symbolic functions for f1 and f2
    function1 = sp.Function('f1')(theta1, theta2, sp.diff(theta1, t), sp.diff(theta2, t))
    function2 = sp.Function('f2')(theta1, theta2, sp.diff(theta1, t), sp.diff(theta2, t))
    
    # Left-hand side of the equation
    LHS = sp.Matrix([[sp.diff(theta1, t, 2)], [sp.diff(theta2, t, 2)]])
    
    # A matrix with symbolic functions alpha1 and alpha2
    A = sp.Matrix([[1, alpha1], [alpha2, 1]])
    
    # Right-hand side of the equation before substitution
    RHS = sp.Matrix([[function1], [function2]])
    
    # Invert matrix A and simplify the product with RHS
    NewRHS = sp.simplify(A.inv() * RHS)
    
    # Create the equation
    EQS = sp.Eq(LHS, (-1) * NewRHS)
    
    # Substitute the derived expressions into the equation
    EQS_subst = EQS.subs({
        alpha1: theta1dd_coeff, 
        alpha2: theta2dd_coeff, 
        function1: rhs_1, 
        function2: rhs_2
    })
    
    RHS_1 = sp.simplify(EQS_subst.rhs[0])
    RHS_2 = sp.simplify(EQS_subst.rhs[1])
    
    return RHS_1, RHS_2

In [26]:
RHS_1, RHS_2 = create_matrix_equation(alpha1, alpha2, function_1, function_2)

In [27]:
display(RHS_1)

(42*M1*g*sin(theta1(t)) + 9*M2*g*sin(theta1(t) - 2*theta2(t)) + 33*M2*g*sin(theta1(t)) + 9*M2*l1*sin(2*theta1(t) - 2*theta2(t))*Derivative(theta1(t), t)**2/2 + 21*M2*l2*sin(theta1(t) - theta2(t))*Derivative(theta2(t), t)**2)/(l1*(-49*M1 + 9*M2*cos(theta1(t) - theta2(t))**2 - 21*M2))

In [28]:
display(RHS_2)

3*((7*M1 + 3*M2)*(2*g*sin(theta2(t)) - l1*sin(theta1(t) - theta2(t))*Derivative(theta1(t), t)**2) - 3*(2*M1*g*sin(theta1(t)) + 2*M2*g*sin(theta1(t)) + M2*l2*sin(theta1(t) - theta2(t))*Derivative(theta2(t), t)**2)*cos(theta1(t) - theta2(t)))/(l2*(-49*M1 + 9*M2*cos(theta1(t) - theta2(t))**2 - 21*M2))

----
&nbsp;
## Rewrite as first order system

Declare new variables $\omega_1$ and $\omega_2$

In [29]:
def first_order_system(RHS_1, RHS_2): 
    """
    Convert a second-order differential equation system to a first-order system in matrix form.

    This function takes the expressions for the right-hand side (RHS) of two second-order 
    differential equations and constructs a first-order system by introducing new functions 
    omega1 and omega2, which represent the first derivatives of the original functions 
    theta1 and theta2, respectively.

    Parameters:
    RHS_1 : sympy.Expr
        The right-hand side expression of the differential equation for d^2(theta1)/dt^2.
    RHS_2 : sympy.Expr
        The right-hand side expression of the differential equation for d^2(theta2)/dt^2.

    Returns:
    tuple: 
        - sympy.Eq representing the matrix equation of the first-order system,
        - sympy.Expr for the derivative of theta1 with respect to time after substitution,
        - sympy.Expr for the derivative of theta2 with respect to time after substitution,
        - sympy.Expr for the derivative of omega1 with respect to time after substitution,
        - sympy.Expr for the derivative of omega2 with respect to time after substitution.

    The matrix equation (MAT_EQ) equates a column matrix of the left-hand sides of the equations 
    to a column matrix of the right-hand sides, facilitating the representation and manipulation 
    of the system of equations.

    The function returns this matrix equation along with the isolated expressions for the first 
    derivatives of theta1 and theta2 (eqn1, eqn2), and the expressions for the second derivatives 
    of omega1 and omega2 (eqn3, eqn4), which correspond to the second derivatives of theta1 and 
    theta2, respectively.
    """
    # Define new functions for the first derivatives of theta1 and theta2
    omega1 = sp.Function('omega1')(t)
    omega2 = sp.Function('omega2')(t)

    # Cast as first order system
    eq1 = sp.Eq(omega1, sp.diff(theta1, t))
    eq2 = sp.Eq(omega2, sp.diff(theta2, t))
    eq3 = sp.Eq(sp.diff(omega1, t), RHS_1)
    eq4 = sp.Eq(sp.diff(omega2, t), RHS_2)

    # Assemble the system into matrix form
    LHS_FIRST = sp.Matrix([[eq1.lhs], [eq2.lhs], [eq3.lhs], [eq4.lhs]])
    RHS_FIRST = sp.Matrix([[eq1.rhs], [eq2.rhs], [eq3.rhs], [eq4.rhs]])

    # Create the matrix equation
    MAT_EQ = sp.Eq(LHS_FIRST, RHS_FIRST)

    # Substitute omega for derivative of theta
    derivative_subs = {sp.Derivative(theta1, t): omega1, sp.Derivative(theta2, t): omega2}

    # Isolate rhs of equations
    eqn1 = eq1.rhs.subs(derivative_subs)
    eqn2 = eq2.rhs.subs(derivative_subs)
    eqn3 = eq3.rhs.subs(derivative_subs)
    eqn4 = eq4.rhs.subs(derivative_subs)

    return MAT_EQ, eqn1, eqn2, eqn3, eqn4

In [30]:
MAT_EQ, eqn1, eqn2, eqn3, eqn4 = first_order_system(RHS_1, RHS_2)

----
&nbsp;
## Derived Quantities

In [31]:
display(MAT_EQ)

Eq(Matrix([
[               omega1(t)],
[               omega2(t)],
[Derivative(omega1(t), t)],
[Derivative(omega2(t), t)]]), Matrix([
[                                                                                                                                                                                                                                                                                   Derivative(theta1(t), t)],
[                                                                                                                                                                                                                                                                                   Derivative(theta2(t), t)],
[                (42*M1*g*sin(theta1(t)) + 9*M2*g*sin(theta1(t) - 2*theta2(t)) + 33*M2*g*sin(theta1(t)) + 9*M2*l1*sin(2*theta1(t) - 2*theta2(t))*Derivative(theta1(t), t)**2/2 + 21*M2*l2*sin(theta1(t) - theta2(t))*Derivative(theta2(t), t)**2)/(l1*(-49*M1 + 9*M

#### Right hand sides have been substituted such that;
$$
\frac{\text{d}}{\text{d} t}\theta_i\equiv\omega_i
$$

In [32]:
display(eqn3)

(42*M1*g*sin(theta1(t)) + 9*M2*g*sin(theta1(t) - 2*theta2(t)) + 33*M2*g*sin(theta1(t)) + 9*M2*l1*omega1(t)**2*sin(2*theta1(t) - 2*theta2(t))/2 + 21*M2*l2*omega2(t)**2*sin(theta1(t) - theta2(t)))/(l1*(-49*M1 + 9*M2*cos(theta1(t) - theta2(t))**2 - 21*M2))

In [33]:
display(eqn4)

3*((7*M1 + 3*M2)*(2*g*sin(theta2(t)) - l1*omega1(t)**2*sin(theta1(t) - theta2(t))) - 3*(2*M1*g*sin(theta1(t)) + 2*M2*g*sin(theta1(t)) + M2*l2*omega2(t)**2*sin(theta1(t) - theta2(t)))*cos(theta1(t) - theta2(t)))/(l2*(-49*M1 + 9*M2*cos(theta1(t) - theta2(t))**2 - 21*M2))

----
&nbsp;
### Numerical Integration in [DoublePendulum Class](DoublePendulum_Compound.py)